# Find and interact with LAMMPS potentials and crystals

This Notebook provides a quick overview of how you can find and interact with interatomic potentials and associated relaxed crystals hosted in the NIST Interatomic Potentials Repository (IRP) using Python.

In [1]:
import atomman as am

## 1. Load LAMMPS potentials

The simplest and easiest way to find and use LAMMPS potentials with atomman is with __atomman.load_lammps_potential()__.  With this one function, you can retrieve a LAMMPS interatomic potential from the IPR database and download any parameter files.

When called, the function will first check a local directory for any matching potentials before querying potentials.nist.gov. If multiple matches are found a prompt will appear asking you to select one.

In [2]:
potential = am.load_lammps_potential(
    #id = '2004--Zhou-X-W--Au--LAMMPS--ipr1',   # The LAMMPS potential id(s) as seen on the IPR website
    pair_style = 'eam/alloy',                   # The LAMMPS pair_style(s) to find
    #symbols = 'Au',                            # symbols are the particle model "names"
    elements = 'Au',                            # elements are the elements that the models represent
    #local = False,                             # Setting to False skips checking the local database copy
    #remote = False,                            # Setting to False skips checking potentials.nist.gov
    getfiles = True,                            # Setting to True copies/downloads parameter files to the working directory  
    verbose = True                              # Setting to True prints informative messages
)

Found 12 matching potential_LAMMPS records in local library
Found 0 matching potential_LAMMPS_KIM records in local library
Built 0 lammps potentials for KIM models
Multiple matching records retrieved from local
1 2004--Zhou-X-W--Au--LAMMPS--ipr1 Au
2 2004--Zhou-X-W--Au--LAMMPS--ipr2 Au
3 2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr1 Cu-Ag-Au
4 2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr2 Cu-Ag-Au
5 2004--Zhou-X-W--Cu-Ag-Au-Ni-Pd-Pt-Al-Pb-Fe-Mo-Ta-W-Mg-Co-Ti-Zr--LAMMPS--ipr1 Cu-Ag-Au-Ni-Pd-Pt-Al-Pb-Fe-Mo-Ta-W-Mg-Co-Ti-Zr
6 2005--Grochola-G--Au--LAMMPS--ipr1 Au
7 2009--Zhakhovskii-V-V--Au--LAMMPS--ipr1 Au
8 2009--Zhakhovskii-V-V--Au--LAMMPS--ipr2 Au
9 2010--Olsson-P-A-T--Au--LAMMPS--ipr1 Au
10 2012--Norman-G-E--Au--LAMMPS--ipr1 Au
11 2017--OBrien-C-J--Pt-Au--LAMMPS--ipr1 Pt-Au
12 2017--Purja-Pun-G-P--Au--LAMMPS--ipr1 Au


Please select one: 1


Au.set copied to 2004--Zhou-X-W--Au--LAMMPS--ipr1


The returned LAMMPS potential object can be used to build the correct LAMMPS command lines for using the potential.

In [3]:
print(potential.pair_info())

print "Parameter file(s) can be downloaded at:"
print "https://www.ctcms.nist.gov/potentials/Download/2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Au/2/Au.set"

pair_style eam/alloy 
pair_coeff * * 2004--Zhou-X-W--Au--LAMMPS--ipr1\Au.set Au

mass 1 196.97




In [4]:
print(potential.pair_info(['Au', 'Au'], comments=False))

pair_style eam/alloy 
pair_coeff * * 2004--Zhou-X-W--Au--LAMMPS--ipr1\Au.set Au Au

mass 1 196.97
mass 2 196.97




## 2. Load relaxed crystal structures

The NIST IPR database has a large number of potential-specific relaxed crystal structures.  These were obtained from relaxing crystal prototype structures as well as crystals from the DFT databases of Materials Project and OQMD.

Relaxed crystal records can easily be retrieved from the IPR database using atomman.load() with the 'crystal' option.

In [5]:
ucell = am.load(
    'crystal',
    family = 'A1--Cu--fcc',                        # Limit by prototype, MP-ID or OQMD-ID
    potential = potential,                         
    #symbols = ['Au'],                             # Limit by atom model symbols used
    #natoms = 4,                                   # Limit by number of atoms
    #natypes = 1,                                  # Limit by number of different atom types
    #local = False,                                # Setting to False skips checking the local database copy
    #remote = False,                               # Setting to False skips checking potentials.nist.gov
    verbose = True                                 # Setting to True prints informative messages
)
print(ucell)

Matching record retrieved from remote
avect =  [ 4.080,  0.000,  0.000]
bvect =  [ 0.000,  4.080,  0.000]
cvect =  [ 0.000,  0.000,  4.080]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Au',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   2.040 |   2.040
      2 |       1 |   2.040 |   0.000 |   2.040
      3 |       1 |   2.040 |   2.040 |   0.000


With an atomic configuration, such as ucell, you can save it to a LAMMPS data file using dump('atom_data').  If you pass in the potential object as well, it will simultaneously generate all LAMMPS commands for loading the configuration file and defining the potential.

In [7]:
load_pair_info = ucell.dump('atom_data', f='ucell.dat', potential=potential)

In [8]:
print(load_pair_info)

units metal
atom_style atomic

boundary p p p
read_data ucell.dat

print "Parameter file(s) can be downloaded at:"
print "https://www.ctcms.nist.gov/potentials/Download/2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Au/2/Au.set"

pair_style eam/alloy 
pair_coeff * * 2004--Zhou-X-W--Au--LAMMPS--ipr1\Au.set Au

mass 1 196.97




In [9]:
with open('ucell.dat') as f:
    print(f.read())


4 atoms
1 atom types
0.0000000000000 4.0797870851023 xlo xhi
0.0000000000000 4.0797870851023 ylo yhi
0.0000000000000 4.0797870851023 zlo zhi

Atoms # atomic

1 1 0.0000000000000 0.0000000000000 0.0000000000000
2 1 0.0000000000000 2.0398935425512 2.0398935425512
3 1 2.0398935425512 0.0000000000000 2.0398935425512
4 1 2.0398935425512 2.0398935425512 0.0000000000000



## 3. More complex database interactions

### 3.1. Defining a Database object

The underlying APIs of atomman.load_lammps_potential() and atomman.load('crystal') are also accessible through atomman.library.Database() allowing for more nuanced interactions. When you initialize an atomman.library.Database() object, you are specifying two database locations to interact with:
- __remote_database__ is a non-local database.  By default, this is potentials.nist.gov.
- __local_database__ is a local database.  This can either be a local directory of files or a locally-hosted MongoDB.  Any queries performed will check for matches here before checking the remote database. By default, this is a subdirectory in your home directory.

Default settings can be changed using atomman.settings.  Different local and remote databases to interact with can also be specified when a atomman.library.Database() is initialized.


In [7]:
potdb = am.library.Database()
print(potdb.remote_database)
print(potdb.local_database)

database style cdcs at https://potentials.nist.gov/
database style local at C:\Users\lmh1\Documents\library


### 3.2. Explore potentials listings

You can use the database object to explore the potentials listings as they appear on the NIST IPR website.

- __get_potentials()__ finds all matching potentials listings and returns them as a list.
- __get_potential()__ finds a matching potentials listing.  If more than one match is found, a prompt will ask you to select it.
- __download_potentials()__ finds all matching potentials in the remote database and copies them to the local database.

In [10]:
listing = potdb.get_potential(elements='Au')

Multiple matching record retrieved from local
1 1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ag-Au-Cu-Ni-Pd-Pt
2 1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Au
3 1987--Ackland-G-J-Tichy-G-Vitek-V-Finnis-M-W--Au
4 1989--Adams-J-B-Foiles-S-M-Wolfer-W-G--Ag-Au-Cu-Ni-Pd-Pt
5 1989--Adams-J-B-Foiles-S-M-Wolfer-W-G--Au
6 1990--Ackland-G-J-Vitek-V--Cu-Ag-Au
7 1996--Jacobsen-K-W-Stoltze-P-Norskov-J-K--Al-Ag-Au-Cu-Ni-Pd-Pt
8 1996--Jacobsen-K-W-Stoltze-P-Norskov-J-K--Au
9 2003--Lee-B-J-Shim-J-H-Baskes-M-I--Au
10 2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Au
11 2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Cu-Ag-Au
12 2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Cu-Ag-Au-Ni-Pd-Pt-Al-Pb-Fe-Mo-Ta-W-Mg-Co-Ti-Zr
13 2005--Grochola-G-Russo-S-P-Snook-I-K--Au
14 2009--Guthikonda-V-S-Elliott-R-S--Au-Cd
15 2009--Zhakhovskii-V-V-Inogamov-N-A-Petrov-Y-V-et-al--Au
16 2010--Olsson-P-A-T--Au
17 2012--Norman-G-E-Starikov-S-V-Stegailov-V-V--Au
18 2015--Elliott-R-S-Akerson-A--Au
19 2017--OBrien-C-J-Barr-C-M-Price-P-M-et-al--Pt-Au
20 2017--P

Please select one: 20


In [11]:
listing.html(render=True)

### 3.3. Explore LAMMPS potentials

- __get_lammps_potentials()__ finds all matching LAMMPS potential records and returns them as a list.
- __get_lammps_potential()__ finds a matching LAMMPS potential record.  If multiple matches are found, a prompt asks you to select one.
- __get_lammps_potential_files()__ copies/downloads any parameter files for a LAMMPS potential from a database to the local directory.
- __download_lammps_potentials()__ finds all matching potentials in the remote database and copies them (and any parameter files) to the local database.


In [12]:
potential = potdb.get_lammps_potential(id='2017--Purja-Pun-G-P--Au--LAMMPS--ipr1')

In [13]:
potential.html(render=True)

In [14]:
print(potential.pair_info())

print "Potential 2017--Purja-Pun-G-P--Au--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:"
print "https://www.ctcms.nist.gov/potentials/entry/2017--Purja-Pun-G-P--Au/2017--Purja-Pun-G-P--Au--LAMMPS--ipr1.html"
print "Parameter file(s) can be downloaded at:"
print "https://www.ctcms.nist.gov/potentials/Download/2017--Purja-Pun-G-P--Au/1/Au_GLJ10_3.eam.alloy"

pair_style eam/alloy 
pair_coeff * * 2017--Purja-Pun-G-P--Au--LAMMPS--ipr1\Au_GLJ10_3.eam.alloy Au

mass 1 196.96654




In [17]:
print(potential.pair_info(comments=False))

pair_style eam/alloy 
pair_coeff * * 2017--Purja-Pun-G-P--Au--LAMMPS--ipr1\Au_GLJ10_3.eam.alloy Au

mass 1 196.96654




### 3.4. Explore crystal structures

The NIST IPR database has a large number of potential-specific relaxed crystal structures.  These were obtained from relaxing crystal prototype structures as well as crystals from the DFT databases of Materials Project and OQMD.

- __get_relaxed_crystals()__ finds all matching relaxed crystal records and returns them as a list.
- __get_relaxed_crystal()__ finds a matching relaxed crystal record. If multiple matches are found, a prompt asks you to select one.
- __download_relaxed_crystals()__ finds all matching relaxed crystals in the remote database and copies them (and any parameter files) to the local database.  *CAUTION!!! There are currently 179,000 relaxed crystal records in the remote database!!!*  


In [19]:
crystal = potdb.get_relaxed_crystal(potential_LAMMPS_id=potential.id, family='A1--Cu--fcc')
print(crystal.ucell)

Multiple matching record retrieved from remote
#  family               symbols  alat    Ecoh    method  standing
 1 A1--Cu--fcc          Au        4.0780 -3.9300 static  bad 
 2 A1--Cu--fcc          Au        4.0780 -3.9300 dynamic good
 3 A1--Cu--fcc          Au        4.0780 -3.9300 box     bad 


Please select one: 2


avect =  [ 4.078,  0.000,  0.000]
bvect =  [ 0.000,  4.078,  0.000]
cvect =  [ 0.000,  0.000,  4.078]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Au',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   2.039 |   2.039
      2 |       1 |   2.039 |   0.000 |   2.039
      3 |       1 |   2.039 |   2.039 |   0.000


## 4. Populating your local database

The Database design mentioned in the last section makes it easy for users to find potentials and structures either from potentials.nist.gov or a local copy.  Having a local copy can be convenient as it can be used offline and can contain extra content not in the IPR database.  To this end, you can copy records to your local database in a few different ways.

- https://github.com/lmhale99/potentials-library contains an archive copy of the potentials listings, LAMMPS potentials and parameter files, crystal prototypes, and defect configuration parameters.  It is an easy way to download all records at once, and the downloaded directory can be set as the local database location.
- The Database.download_\*() methods mentioned above can retrieve the individual record types.  Alternatively, you can call Database.download_all() to get all current potentials listings, LAMMPS potentials and parameter files, and crystal prototypes.  

I do tend to forget to push updates for the potentials-library github, so the best method is probably to download from there first, then call download_all(overwrite=False).